# Stacking Models

<table align="left"><td>
  <a target="_blank"  href="https://colab.research.google.com/github/TannerGilbert/Tutorials/blob/master/A%20guide%20to%20Ensemble%C2%A0Learning/A%20guide%20to%20Ensemble%20Learning.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab
  </a>
</td><td>
  <a target="_blank"  href="https://github.com/TannerGilbert/Tutorials/blob/master/A%20guide%20to%20Ensemble%C2%A0Learning/A%20guide%20to%20Ensemble%20Learning.ipynb">
    <img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
</td></table>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pathlib
from scipy import stats
from scipy.stats import norm, skew

from sklearn.linear_model import Lasso
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.tree import DecisionTreeRegressor

import xgboost as xgb
import lightgbm as lgb

import warnings
warnings.filterwarnings("ignore")

C:\Users\Gilbert\AppData\Local\Continuum\anaconda3\envs\ml\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


## Load in data-sets

In [2]:
path = pathlib.Path('C:/Users/Gilbert/Desktop/Programming/Datasets/House Prices advanced regression techniques')

In [3]:
# Load in the train and test dataset
train = pd.read_csv(path/'train.csv')
test = pd.read_csv(path/'test.csv')

train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
train.isnull().sum().sort_values(ascending=False)

PoolQC           1453
MiscFeature      1406
Alley            1369
Fence            1179
FireplaceQu       690
LotFrontage       259
GarageCond         81
GarageType         81
GarageYrBlt        81
GarageFinish       81
GarageQual         81
BsmtExposure       38
BsmtFinType2       38
BsmtFinType1       37
BsmtCond           37
BsmtQual           37
MasVnrArea          8
MasVnrType          8
Electrical          1
Utilities           0
YearRemodAdd        0
MSSubClass          0
Foundation          0
ExterCond           0
ExterQual           0
Exterior2nd         0
Exterior1st         0
RoofMatl            0
RoofStyle           0
YearBuilt           0
                 ... 
GarageArea          0
PavedDrive          0
WoodDeckSF          0
OpenPorchSF         0
3SsnPorch           0
BsmtUnfSF           0
ScreenPorch         0
PoolArea            0
MiscVal             0
MoSold              0
YrSold              0
SaleType            0
Functional          0
TotRmsAbvGrd        0
KitchenQua

In [5]:
# Save the 'Id' column
train_ID = train['Id']
test_ID = test['Id']

# Drop id column because is is unnecessary for making predictions
train.drop('Id', axis=1, inplace=True)
test.drop('Id', axis=1, inplace=True)

## Feature Engineering
Based on https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard

In [6]:
train['SalePrice'] = np.log1p(train['SalePrice'])

In [7]:
ntrain = train.shape[0]
ntest = test.shape[0]
y_train = train.SalePrice.values
all_data = pd.concat((train, test)).reset_index(drop=True)
all_data.drop(['SalePrice'], axis=1, inplace=True)
print('all_data size is: {}'.format(all_data.shape))

all_data size is: (2919, 79)


### Missing data

In [8]:
all_data_na = (all_data.isnull().sum() / len(all_data))*100
all_data_na = all_data_na.drop(all_data_na[all_data_na==0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ration': all_data_na})
missing_data.head(20)

,Missing Ration
PoolQC,99.657417
MiscFeature,96.402878
Alley,93.216855
Fence,80.438506
FireplaceQu,48.646797
LotFrontage,16.649538
GarageQual,5.447071
GarageCond,5.447071
GarageFinish,5.447071
GarageYrBlt,5.447071


### Imputing missing values

In [9]:
all_data['PoolQC'] = all_data['PoolQC'].fillna('None')
all_data['MiscFeature'] = all_data['MiscFeature'].fillna('None')
all_data['Alley'] = all_data['Alley'].fillna('None')
all_data['Fence'] = all_data['Fence'].fillna('None')
all_data['FireplaceQu'] = all_data['FireplaceQu'].fillna('None')
all_data['LotFrontage'] = all_data.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    all_data[col] = all_data[col].fillna('None')
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    all_data[col] = all_data[col].fillna(0)
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    all_data[col] = all_data[col].fillna(0)
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    all_data[col] = all_data[col].fillna('None')
all_data['MasVnrType'] = all_data['MasVnrType'].fillna('None')
all_data['MasVnrArea'] = all_data['MasVnrArea'].fillna(0)
all_data['MSZoning'] = all_data['MSZoning'].fillna(all_data['MSZoning'].mode()[0])
all_data = all_data.drop(['Utilities'], axis=1)
all_data['Functional'] = all_data['Functional'].fillna('Typ')
all_data['Electrical'] = all_data['Electrical'].fillna(all_data['Electrical'].mode()[0])
all_data['KitchenQual'] = all_data['KitchenQual'].fillna(all_data['KitchenQual'].mode()[0])
all_data['Exterior1st'] = all_data['Exterior1st'].fillna(all_data['Exterior1st'].mode()[0])
all_data['Exterior2nd'] = all_data['Exterior2nd'].fillna(all_data['Exterior2nd'].mode()[0])
all_data['SaleType'] = all_data['SaleType'].fillna(all_data['SaleType'].mode()[0])
all_data['MSSubClass'] = all_data['MSSubClass'].fillna('None')

In [10]:
all_data_na = (all_data.isnull().sum() / len(all_data))*100
all_data_na = all_data_na.drop(all_data_na[all_data_na==0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio':all_data_na})
missing_data.head()

,Missing Ratio


In [11]:
train = all_data[:ntrain]
test = all_data[ntrain:]
train.to_csv(path/'train_cleaned.csv')
test.to_csv(path/'test_cleaned.csv')

### Transforming some numerical features to categorical

In [12]:
all_data['MSSubClass'] = all_data['MSSubClass'].apply(str)

all_data['OverallCond'] = all_data['OverallCond'].astype(str)

all_data['YrSold'] = all_data['YrSold'].astype(str)
all_data['MoSold'] = all_data['MoSold'].astype(str)

In [13]:
from sklearn.preprocessing import LabelEncoder
cols = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold')

for c in cols:
    lbl = LabelEncoder()
    lbl.fit(list(all_data[c].values))
    all_data[c] = lbl.transform(list(all_data[c].values))
    
print('Shape all_data {}'.format(all_data.shape))

Shape all_data (2919, 78)


In [14]:
all_data['TotalSF'] = all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']

In [15]:
numeric_feats = all_data.dtypes[all_data.dtypes!='object'].index

# Check the skew of all the numeric features
skewed_feats = all_data[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print('Skew in numeric features:')
skewness = pd.DataFrame({'Skew': skewed_feats})
skewness.head(10)

Skew in numeric features:


,Skew
MiscVal,21.947195
PoolArea,16.898328
LotArea,12.822431
LowQualFinSF,12.088761
3SsnPorch,11.376065
LandSlope,4.975157
KitchenAbvGr,4.302254
BsmtFinSF2,4.146143
EnclosedPorch,4.003891
ScreenPorch,3.946694


In [16]:
skewness = skewness[abs(skewness) > 0.75]
print('There are {} skewed numerical features to Box Cox transform'.format(skewness.shape[0]))

from scipy.special import boxcox1p
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    all_data[feat] = boxcox1p(all_data[feat], lam)

There are 59 skewed numerical features to Box Cox transform


In [17]:
print(all_data.shape)
all_data = pd.get_dummies(all_data)
print(all_data.shape)

(2919, 79)
(2919, 221)


In [18]:
train = all_data[:ntrain]
test = all_data[ntrain:]

In [19]:
train.head()

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,...,SaleCondition_Partial,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD
0,11.692623,11.686189,0.0,0.730463,1.540963,1.820334,1.540963,11.170327,0.0,1.194318,...,0,0,0,0,0,0,0,0,0,1
1,12.792276,0.000000,0.0,0.730463,1.540963,1.820334,0.730463,12.062832,0.0,0.000000,...,0,0,0,0,0,0,0,0,0,1
2,11.892039,11.724598,0.0,0.730463,1.540963,1.820334,1.194318,10.200343,0.0,1.194318,...,0,0,0,0,0,0,0,0,0,1
3,12.013683,11.354094,0.0,0.730463,1.540963,0.730463,1.540963,8.274266,0.0,0.000000,...,0,0,0,0,0,0,0,0,0,1
4,12.510588,12.271365,0.0,0.730463,1.820334,1.820334,0.000000,10.971129,0.0,1.194318,...,0,0,0,0,0,0,0,0,0,1


In [20]:
train.dtypes.unique()

array([dtype('float64'), dtype('uint8')], dtype=object)

In [21]:
y_train

array([12.24769912, 12.10901644, 12.31717117, ..., 12.49313327,
       11.86446927, 11.90159023])

In [22]:
X = train.values

## Accuracy function

In [23]:
from sklearn.model_selection import KFold, cross_val_score

n_folds = 5

def get_cv_scores(model, X, y, print_scores=True):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X)
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv = kf))
    if print_scores:
        print(f'Root mean squared error: {rmse.mean():.3f} ({rmse.std():.3f})')
    return [rmse]

## Base Models

### Lasso

In [24]:
lasso_model = make_pipeline(RobustScaler(), Lasso(alpha=0.0005, random_state=1))

In [25]:
%%time
get_cv_scores(lasso_model, X, y_train)

Root mean squared error: 0.124 (0.016)
Wall time: 855 ms


[array([0.1036135 , 0.13480558, 0.12784684, 0.10698948, 0.14677378])]

In [26]:
lasso_model.fit(X, y_train)
lasso_model.predict([X[0]])

array([12.23428681])

In [27]:
y_train[0]

12.24769911637256

In [28]:
lasso_model.score(X, y_train)

0.9285256686605037

### DecisionTreeRegresor

In [29]:
%%time
dt = DecisionTreeRegressor()
get_cv_scores(dt, X, y_train);

Root mean squared error: 0.210 (0.014)
Wall time: 306 ms


### RandomForestRegressor

In [30]:
rf = RandomForestRegressor()

In [31]:
%%time
get_cv_scores(rf, X, y_train);

Root mean squared error: 0.151 (0.010)
Wall time: 1.84 s


[array([0.14088252, 0.16324013, 0.15008495, 0.14033984, 0.16150976])]

### GradientBoostingRegressor

In [32]:
gbr = GradientBoostingRegressor()

In [33]:
%%time
get_cv_scores(gbr, X, y_train);

Root mean squared error: 0.125 (0.008)
Wall time: 5.13 s


[array([0.11485383, 0.13546331, 0.13424406, 0.11705409, 0.12451019])]

In [34]:
gbr = GradientBoostingRegressor()
gbr.fit(X, y_train)
gbr.score(X, y_train)

0.9581780008371052

### XGBoost

In [35]:
xgb_model = xgb.XGBRegressor()

In [36]:
%%time
get_cv_scores(xgb_model, X, y_train);

Root mean squared error: 0.128 (0.008)
Wall time: 2.96 s


[array([0.1191355 , 0.14068314, 0.13207005, 0.12101365, 0.12790841])]

### LightGBM

In [37]:
lgb_model = lgb.LGBMRegressor()

In [38]:
%%time
get_cv_scores(lgb_model, X, y_train);

Root mean squared error: 0.133 (0.009)
Wall time: 2.29 s


[array([0.12479658, 0.1460139 , 0.13846976, 0.12297723, 0.13329413])]

## Stacking Models

In [39]:
from sklearn.base import BaseEstimator, TransformerMixin, clone, RegressorMixin

In [40]:
# based on https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard
# and https://www.kaggle.com/eikedehling/trying-out-stacking-approaches
class AveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)

In [41]:
class WeightedAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models, weights):
        self.models = models
        self.weights = weights
        assert sum(self.weights)==1
        
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.sum(predictions*self.weights, axis=1)

In [42]:
class BaggingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]

        for model in self.models_:
            X_tmp, y_tmp = self.subsample(X, y)
            model.fit(X_tmp, y_tmp)
        
        return self
            
    # Create a random subsample from the dataset with replacement
    def subsample(self, X, y, ratio=1.0):
        X_new, y_new = list(), list()
        n_sample = round(len(X) * ratio)
        while len(X_new) < n_sample:
            index = np.random.randint(len(X))
            X_new.append(X[index])
            y_new.append(y[index])
        return X_new, y_new
    
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)

In [43]:
#class BoostingModels(BaseEstimator, RegressorMixin, TransformerMixin):
#    def __init__(self, base_models)

In [44]:
# based on https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard
# and https://www.kaggle.com/eikedehling/trying-out-stacking-approaches
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5, use_features_in_secondary=False):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
        self.use_features_in_secondary = use_features_in_secondary
        
    def fit(self, X, y):
        """Fit all the models on the given dataset"""
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=42)
        
        # Train cloned base models and create out-of-fold predictions
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
        
        if self.use_features_in_secondary:
            self.meta_model_.fit(np.hstack((X, out_of_fold_predictions)), y)
        else:
            self.meta_model_.fit(out_of_fold_predictions, y)
            
        return self
    
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        if self.use_features_in_secondary:
            return self.meta_model_.predict(np.hstack((X, meta_features)))
        else:
            return self.meta_model_.predict(meta_features)
    
    def predict_proba(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        if self.use_features_in_secondary:
            return self.meta_model_.predict_proba(np.hstack((X, meta_features)))
        else:
            return self.meta_model_.predict_proba(meta_features)

In [45]:
# based on https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard
# and https://www.kaggle.com/eikedehling/trying-out-stacking-approaches
class StackingModelsRetrained(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5, use_features_in_secondary=False):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
        self.use_features_in_secondary = use_features_in_secondary
        
    def fit(self, X, y):
        """Fit all the models on the given dataset"""
        self.base_models_ = [clone(x) for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=42)
        
        # Train cloned base models and create out-of-fold predictions
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
        
        if self.use_features_in_secondary:
            self.meta_model_.fit(np.hstack((X, out_of_fold_predictions)), y)
        else:
            self.meta_model_.fit(out_of_fold_predictions, y)
            
        for model in self.base_models_:
            model.fit(X, y)
            
        return self
    
    def predict(self, X):
        meta_features = np.column_stack([
            base_model.predict(X) for base_model in self.base_models_ ])
        if self.use_features_in_secondary:
            return self.meta_model_.predict(np.hstack((X, meta_features)))
        else:
            return self.meta_model_.predict(meta_features)
    
    def predict_proba(self, X):
        meta_features = np.column_stack([
            base_model.predict(X) for base_model in self.base_models_ ])
        if self.use_features_in_secondary:
            return self.meta_model_.predict_proba(np.hstack((X, meta_features)))
        else:
            return self.meta_model_.predict_proba(meta_features)

In [46]:
from sklearn.model_selection import train_test_split

class BlendingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, holdout_pct=0.2, use_features_in_secondary=False):
        self.base_models = base_models
        self.meta_model = meta_model
        self.holdout_pct = holdout_pct
        self.use_features_in_secondary = use_features_in_secondary
        
    def fit(self, X, y):
        """Fit all the models on the given dataset"""
        self.base_models_ = [clone(x) for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        
        X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=self.holdout_pct)
                
        holdout_predictions = np.zeros((X_holdout.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models_):
            model.fit(X_train, y_train)
            y_pred = model.predict(X_holdout)
            holdout_predictions[:, i] = y_pred
        if self.use_features_in_secondary:
            self.meta_model_.fit(np.hstack((X_holdout, holdout_predictions)), y_holdout)
        else:
            self.meta_model_.fit(holdout_predictions, y_holdout)
            
        return self
    
    def predict(self, X):
        meta_features = np.column_stack([
            model.predict(X) for model in self.base_models_
        ])
        if self.use_features_in_secondary:
            return self.meta_model_.predict(np.hstack((X, meta_features)))
        else:
            return self.meta_model_.predict(meta_features)

## Trying Stacking with different Sub- and Aggregating Models

In [47]:
%%time
averaged_model1 = AveragedModels([gbr, lasso_model, xgb_model])
get_cv_scores(averaged_model1, X, y_train);

Root mean squared error: 0.119 (0.009)
Wall time: 13.4 s


In [48]:
%%time
averaged_model1 = AveragedModels([gbr, lasso_model, xgb_model, lgb_model])
get_cv_scores(averaged_model1, X, y_train);

Root mean squared error: 0.120 (0.009)
Wall time: 16.9 s


In [49]:
%%time
weighted_average_model = WeightedAveragedModels([gbr, lasso_model, xgb_model], [0.3, 0.3, 0.4])
get_cv_scores(weighted_average_model, X, y_train);

Root mean squared error: 0.119 (0.009)
Wall time: 13.7 s


In [50]:
%%time
weighted_average_model = WeightedAveragedModels([gbr, lasso_model, xgb_model], [0.3, 0.45, 0.25])
get_cv_scores(weighted_average_model, X, y_train);

Root mean squared error: 0.118 (0.010)
Wall time: 9.51 s


In [51]:
%%time
bagging_model = BaggingModels([gbr, lasso_model, xgb_model])
get_cv_scores(bagging_model, X, y_train);

Root mean squared error: 0.151 (0.016)
Wall time: 9.64 s


In [52]:
%%time
bagging_model = BaggingModels([gbr, lasso_model, xgb_model]*2)
get_cv_scores(bagging_model, X, y_train);

Root mean squared error: 0.152 (0.014)
Wall time: 22.6 s


In [53]:
%%time
bagging_model = BaggingModels([gbr, lasso_model, xgb_model]*3)
get_cv_scores(bagging_model, X, y_train);

Root mean squared error: 0.140 (0.011)
Wall time: 30.3 s


In [54]:
%%time
stacking_model1 = StackingAveragedModels([gbr, lgb_model, xgb_model], lasso_model)
get_cv_scores(stacking_model1, X, y_train);

Root mean squared error: 0.124 (0.009)
Wall time: 45.6 s


In [55]:
%%time
stacking_model2= StackingAveragedModels([gbr, lgb_model, xgb_model], lasso_model, use_features_in_secondary=True)
get_cv_scores(stacking_model2, X, y_train);

Root mean squared error: 0.120 (0.014)
Wall time: 43.7 s


In [56]:
%%time
stacking_model3 = StackingAveragedModels([gbr, lgb_model, xgb_model, lasso_model], lasso_model)
get_cv_scores(stacking_model3, X, y_train);

Root mean squared error: 0.119 (0.014)
Wall time: 48.2 s


In [57]:
%%time
stacking_model4 = StackingAveragedModels([gbr, lgb_model, xgb_model, lasso_model], lasso_model, use_features_in_secondary=True)
get_cv_scores(stacking_model4, X, y_train);

Root mean squared error: 0.120 (0.014)
Wall time: 58.8 s


In [58]:
%%time
blending_model1 = BlendingAveragedModels([gbr, lgb_model, xgb_model, lasso_model], lasso_model)
get_cv_scores(blending_model1, X, y_train);

Root mean squared error: 0.125 (0.011)
Wall time: 10.2 s


In [59]:
%%time
blending_model2 = BlendingAveragedModels([gbr, lgb_model, xgb_model, lasso_model], lasso_model, use_features_in_secondary=True)
get_cv_scores(blending_model2, X, y_train);

Root mean squared error: 0.142 (0.014)
Wall time: 10.5 s


In [60]:
%%time
stacking_retrained_model1 = StackingModelsRetrained([gbr, lgb_model, xgb_model], lasso_model)
get_cv_scores(stacking_retrained_model1, X, y_train);

Root mean squared error: 0.125 (0.008)
Wall time: 45.7 s


In [61]:
%%time
stacking_retrained_model2 = StackingModelsRetrained([gbr, lgb_model, xgb_model], lasso_model, use_features_in_secondary=True)
get_cv_scores(stacking_retrained_model2, X, y_train);

Root mean squared error: 0.120 (0.014)
Wall time: 49.7 s


In [62]:
%%time
stacking_retrained_model3 = StackingModelsRetrained([gbr, lgb_model, xgb_model, lasso_model], lasso_model)
get_cv_scores(stacking_retrained_model1, X, y_train);

Root mean squared error: 0.125 (0.008)
Wall time: 1min 2s


In [63]:
%%time
stacking_retrained_model4 = StackingModelsRetrained([gbr, lgb_model, xgb_model, lasso_model], lasso_model, use_features_in_secondary=True)
get_cv_scores(stacking_retrained_model2, X, y_train);

Root mean squared error: 0.120 (0.014)
Wall time: 51.4 s
